<a href="https://colab.research.google.com/github/pranay335/SKINAID-Skin.Disease.Classifier.with.Chatbot/blob/main/RagChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install -q -U langchain langchain-community langchain-google-genai chromadb pypdf

In [12]:
import os
# New import path for the splitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Load your dermatology PDF
# Make sure this path is exactly where your file is in the Colab sidebar
file_path = "/content/dermatology_guide__amended_may_2012_-1.pdf"

try:
    loader = PyPDFLoader(file_path)
    data = loader.load()

    # 2. Split the intelligence into chunks
    # We use RecursiveCharacterTextSplitter because it's smart about not
    # cutting middle of sentences.
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(data)

    print(f"✅ Mission Success: {len(docs)} intelligence chunks created!")
except Exception as e:
    print(f"❌ Error: {e}. Check if the PDF file is uploaded to the 'content' folder.")

✅ Mission Success: 128 intelligence chunks created!


In [33]:
import os
from google.colab import userdata

# It will ask you to enter your key here
os.environ["GOOGLE_API_KEY"] = input("Enter your Gemini API Key: ")

In [19]:
!pip install -q -U sentence-transformers langchain-huggingface

In [20]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# 1. Initialize the Local Embedding Model (No API Key needed!)
# This model runs directly in your Colab environment
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 2. Create the Vector Store
# 'docs' are the chunks from your PDF you created earlier
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="./skin_db_local"
)

print("✅ Success! Your medical data is now stored locally. No more quota errors!")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Success! Your medical data is now stored locally. No more quota errors!


In [28]:
!pip install langchain-classic

In [40]:
# 1. Update the import to use the 'classic' package
from langchain_classic.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI

# 2. Initialize the "Brain" (Gemini)
# We use temperature 0.1 for precise medical answers
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1)

# 3. Create the Retrieval Chain
# This connects your Gemini 'Brain' to your ChromaDB 'Memory'
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3})
)

# 4. Ask the Spy a question from the PDF
query = "What is Chiken Pox"
response = qa_chain.invoke(query)

print("\n--- Intelligence Report ---")
print(response["result"])


--- Intelligence Report ---
Chickenpox (Varicella) is caused by the Varicella zoster virus. It commonly occurs in childhood, with most children infected by age 10. The virus remains dormant after primary infection and can reactivate as shingles.

Symptoms include blister-like lesions with a pus-filled center, appearing on the face, scalp, or trunk. Chickenpox is highly contagious and spreads via coughing or sneezing. Complications can include bacterial skin infection, brain swelling, and pneumonia.

While healthy children usually don't require antiviral agents, adults or immunocompromised patients may be prescribed aciclovir, valaciclovir, or famciclovir to reduce the severity of the attack, as adults can become very unwell with more severe symptoms.
